**Ex. 1** În documentaţia pgmpy găsiţi formularea şi implementarea celebrei probleme Monty Hall. Cerinţa acestui exerciţiu
este să explicaţi problema şi codul din implementare.

In [ ]:
import random

def monty_hall_simulation(switch: bool) -> bool:
    # Alegem aleatoriu usa cu premiul
    prize_door = random.randint(1, 3)
    # Concursantul alege o usa aleatoriu
    contestant_choice = random.randint(1, 3)

    # Gazda alege o usa de deschis, care are o capra
    possible_doors = [door for door in [1, 2, 3] if door != contestant_choice and door != prize_door]
    opened_door = random.choice(possible_doors)

    # Daca concursantul schimba, alegem cealalta usa ramasa
    if switch:
        contestant_choice = [door for door in [1, 2, 3] if door != contestant_choice and door != opened_door][0]

    # Returnam rezultatul de castig sau esec a concursantului
    return contestant_choice == prize_door


def monty_hall_experiment(num_trials: int, switch: bool) -> float:
    wins = sum(monty_hall_simulation(switch) for _ in range(num_trials))
    return wins / num_trials

# Simulam 10,000 de jocuri si calculam probabilitatea de castig
num_trials = 10000
print("Probabilitatea de castig fara schimbare:", monty_hall_experiment(num_trials, switch=False))
print("Probabilitatea de castig cu schimbare:", monty_hall_experiment(num_trials, switch=True))


Probabilitatea de castig fara schimbare: 0.3224
Probabilitatea de castig cu schimbare: 0.6706


**Ex. 2** Un medic doreşte să determine probabilitatea ca un pacient să aibă o anumită boală pulmonară (B), având în vedere
simptomele observate. Există trei variabile observabile care pot indica prezenţa bolii: tuse (T), dificultate de respiraţie
(D), radiografie anormală (X). Toate acestea sunt variabile binare.
De asemenea, ştim că:
● Dacă pacientul are boala (B), există o probabilitate ridicată să aibă tuse şi să apară anomalii la radiografie.
● Dificultatea de respiraţie (D) este dependentă atât de prezenţa bolii pulmonare (B), cât şi de prezenţa tusei (T).
Astfel, datele de probabilitate condiţionată sunt următoarele:
1. Probabilitatea apriori pentru boala pulmonară:

P(B = 1) = 0.1, P(B = 0) = 0.9;
2. Probabilitatea condiţionată pentru tuse (T) condiţionată de boală (B):
P(T = 1∣B = 1) = 0.8, P(T = 1∣B = 0) = 0.3;
3. Probabilitatea condiţionată pentru radiografie anormală (X) condiţionată de boală (B):
P(X = 1∣B = 1) = 0.9, P(X = 1∣B = 0) = 0.1;

4. Probabilitatea condiţionată pentru dificultate de respiraţie (D), condiţionată de boală (B) şi tuse (T):

P(D = 1∣B = 1,T = 1) = 0.9, P(D = 1∣B = 1,T = 0) = 0.4;
P(D = 1∣B = 0,T = 1) = 0.5, P(D = 1∣B = 0,T = 0) = 0.1.

1. Definiti modelul probabilist, folosind pgmpy, care sa descrie contextul de mai sus.
2. Medicul observă că pacientul are tuse şi dificultăţi de respiraţie. Determinaţi probabilitatea ca pacientul să aibă
boala pulmonară.

3. Ulterior se descoperă (cu alte mijloace) că pacientul nu avea boala pulmonară. Care este probabilitatea ca radio-
grafia să fi fost anormală?

4. Bonus: Calculaţi cele 2 probabilităţi folosind regula lui Bayes şi incărcaţi calculul fie în Markdown/Latex, fie ca foto

In [ ]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.2 MB/s eta 0:00:00


In [ ]:
# Importuri necesare
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Definim structura retelei
model = BayesianNetwork([
    ('B', 'T'),  # Boala afecteaza Tusea
    ('B', 'X'),  # Boala afecteaza Radiografia anormala
    ('B', 'D'),  # Boala afecteaza Dificultatea de respiratie
    ('T', 'D')   # Tusea afecteaza Dificultatea de respiratie
])

# Definim CPD-urile (distributiile conditionate)
# CPD pentru Boala (B)
cpd_b = TabularCPD(variable='B', variable_card=2, values=[[0.9], [0.1]])

# CPD pentru Tuse (T) conditionata de Boala (B)
cpd_t = TabularCPD(variable='T', variable_card=2,
                   values=[[0.7, 0.2],  # P(T = 0 | B)
                           [0.3, 0.8]], # P(T = 1 | B)
                   evidence=['B'], evidence_card=[2])

# CPD pentru Radiografia anormala (X) conditionata de Boala (B)
cpd_x = TabularCPD(variable='X', variable_card=2,
                   values=[[0.9, 0.1],  # P(X = 0 | B)
                           [0.1, 0.9]], # P(X = 1 | B)
                   evidence=['B'], evidence_card=[2])

# CPD pentru Dificultatea de respiratie (D), conditionata de Boala (B) si Tuse (T)
cpd_d = TabularCPD(variable='D', variable_card=2,
                   values=[[0.1, 0.5, 0.6, 0.1],  # P(D = 0 | B, T)
                           [0.9, 0.5, 0.4, 0.9]], # P(D = 1 | B, T)
                   evidence=['B', 'T'], evidence_card=[2, 2])

# Adaugam CPD-urile in model
model.add_cpds(cpd_b, cpd_t, cpd_x, cpd_d)

# Verificam daca modelul este corect definit
assert model.check_model()

# Definim un obiect pentru inferenta
infer = VariableElimination(model)

# 2. Calculam probabilitatea ca pacientul sa aiba boala (B = 1) stiind ca are tuse (T = 1) si dificultati de respiratie (D = 1)
prob_b_given_td = infer.query(variables=['B'], evidence={'T': 1, 'D': 1})
print("Probabilitatea ca pacientul sa aiba boala, stiind ca are tuse si dificultati de respiratie:", prob_b_given_td)

# 3. Calculam probabilitatea ca radiografia sa fie anormala (X = 1) stiind ca pacientul nu are boala (B = 0)
prob_x_given_b0 = infer.query(variables=['X'], evidence={'B': 0})
print("Probabilitatea ca radiografia sa fie anormala, stiind ca pacientul nu are boala:", prob_x_given_b0)


Probabilitatea ca pacientul sa aiba boala, stiind ca are tuse si dificultati de respiratie: +------+----------+
| B    |   phi(B) |
+======+==========+
| B(0) |   0.6522 |
+------+----------+
| B(1) |   0.3478 |
+------+----------+
Probabilitatea ca radiografia sa fie anormala, stiind ca pacientul nu are boala: +------+----------+
| X    |   phi(X) |
+======+==========+
| X(0) |   0.9000 |
+------+----------+
| X(1) |   0.1000 |
+------+----------+


**Ex. 3** Un joc între doi jucători, J0 şi J1, se desfăşoară în felul următor:
● se aruncă mai întâi cu o monedă (normală) pentru a decide cine începe: J0 sau J1;
● în prima rundă, jucătorul desemnat aruncă cu propriul zar; fie n numărul obţinut;
● în a doua rundă, celălalt jucător aruncă cu moneda proprie de 2n ori; fie m numărul de steme obţinute.
Jucătorul din prima rundă câştigă dacă n ≥ m, în caz contrar câştigând jucătorul din a doua rundă. Mai ştim că
jucătorul J1 este necinstit, el aducând o monedă măsluită, cu probabilitatea de obţinere a stemei egală cu 4/7. În schimb,
moneda jucătorului J0 este normală, iar cele două zaruri sunt şi ele normale.
1. Estimaţi care dintre cei doi jucători are şansele cele mai mari de câştig, simulând un joc de 10000 ori.
2. Folosind pgmpy, definiţi o reţea Bayesiană care sa descrie contextul de mai sus.
3. Folosind modelul de mai sus, determinaţi cine e cel mai probabil să fi început jocul, ştiind că în a doua rundă s-a
obţinut o singură stemă.

In [ ]:
#1 Simularea jocului de 10.000 de ori

import numpy as np

# Setam numarul de simulari
num_simulations = 10000
j0_wins = 0
j1_wins = 0

for _ in range(num_simulations):
    # Etapa 1: Determinarea jucatorului care incepe (aruncare moneda normala)
    starter = np.random.choice(['J0', 'J1'])

    # Etapa 2: Jucatorul care incepe arunca zarul (zar normal de la 1 la 6)
    n = np.random.randint(1, 7)

    # Etapa 3: Celalalt jucator arunca moneda de 2*n ori
    if starter == 'J0':
        # J1 arunca moneda masluita (probabilitate 4/7 pentru stema)
        m = np.sum(np.random.rand(2 * n) < (4 / 7))
    else:
        # J0 arunca moneda normala (probabilitate 1/2 pentru stema)
        m = np.sum(np.random.rand(2 * n) < 0.5)

    # Determinarea castigatorului
    if n >= m:
        if starter == 'J0':
            j0_wins += 1
        else:
            j1_wins += 1
    else:
        if starter == 'J0':
            j1_wins += 1
        else:
            j0_wins += 1

# Calcularea si afisarea rezultatelor
print(f"J0 wins: {j0_wins / num_simulations * 100:.2f}%")
print(f"J1 wins: {j1_wins / num_simulations * 100:.2f}%")

J0 wins: 42.17%
J1 wins: 57.83%


In [ ]:
# 2 -  Definirea rețelei Bayesiene folosind pgmpy
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

# Definirea structurii modelului
model = BayesianNetwork([
    ('Starter', 'M'),  # Starter influenteaza M (tipul de moneda)
    ('N', 'M')         # N influenteaza M (numarul de aruncari)
])

# CPD pentru Starter
cpd_starter = TabularCPD(variable='Starter', variable_card=2,
                         values=[[0.5], [0.5]],
                         state_names={'Starter': ['J0', 'J1']})

# CPD pentru N (valoare zar), independent de cine a inceput
cpd_n = TabularCPD(variable='N', variable_card=6,
                   values=[[1/6] * 6],
                   state_names={'N': [1, 2, 3, 4, 5, 6]})

# CPD pentru M (numar de steme), conditionat de Starter si de N
# Probabilitatile pentru M variaza in functie de cine incepe (J0 sau J1) si valoarea lui N
# Numaram posibilitatile pentru fiecare combinatie de Starter si N
cpd_m = TabularCPD(variable='M', variable_card=13,
                   values=[
                       # Probabilitati pentru J0 (N=1 la N=6)
                       [0.5**k * (0.5**(2*n - k)) for k in range(2*n + 1)]
                       for n in range(1, 7)
                   ] + [
                       # Probabilitati pentru J1 (N=1 la N=6)
                       [(4/7)**k * ((3/7)**(2*n - k)) for k in range(2*n + 1)]
                       for n in range(1, 7)
                   ],
                   evidence=['Starter', 'N'], evidence_card=[2, 6])

# Adaugam CPD-urile in model
model.add_cpds(cpd_starter, cpd_n, cpd_m)

# Verificam modelul
assert model.check_model()

In [ ]:
#3
from pgmpy.inference import VariableElimination

# Initializam inferenta
infer = VariableElimination(model)

# Calculam probabilitatea cine a inceput jocul, stiind ca M = 1
posterior = infer.query(variables=['Starter'], evidence={'M': 1})

# Afisam probabilitatile pentru fiecare jucator
print("Probabilitatea ca J0 sa fi inceput:", posterior.values[0])
print("Probabilitatea ca J1 sa fi inceput:", posterior.values[1])